## Testing the Iris classification trained model

Determined manages the model artifacts, so after model training is completed you can easily retrieve the best performing checkpoint of the model, and use it to make inferences. 

* ***Inferences:*** The process of using a trained model and new unlabelled data to make a prediction.

In addition to Determined CLI commands, Determined provides a Python API to interact with Determine and let you download the best model checkpoint files, and load it into memory in a Python process, for TensorFlow and Pythorch based models.

Let's use the Determined Python API from within the JupyterLab server instance you have just deployed on Determined system and test the model on three unlabelled examples to predict the Iris flower species based on the length and width of their sepal and petal, and validate the model works as expected.

>***Note:*** _To learn more about the Python API, check out the documentation [here](https://docs.determined.ai/latest/interact/api-experimental-client.html#)._

### Import the Python libraries

In [ ]:
# Hide the GPU to run code on CPU and disable all logging output from TensorFlow Keras. 
# We do it before importing keras to avoid library version issue with GPUs
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#
import numpy as np
from determined.experimental import client
from determined import keras
#
determined_master="http://MASTERURL"
myexpId="EXPID"

### Login to Determined System

To interact with Determined, you must first authenticate to Determined system.

In [ ]:
#
# Login to Determined system
#
client.login(master=determined_master, user="USERNAME", password="PASSW")

### Download the best model checkpoint from the shared checkpoint storage

In [ ]:
#
# download the best model checkpoint from the shared checkpoint storage
#
best_checkpoint = client.get_experiment(myexpId).top_checkpoint()
print(f"The best checkpoint for the Experiment {myexpId} is {best_checkpoint.uuid}")
print ("")
print(f"Downloading top checkpoint from experiment: {myexpId}")
checkpoint_path = best_checkpoint.download()
print(f"Downloaded checkpoint to: {checkpoint_path}")

### Loading the model

In [ ]:
#
# load the best model checkpoint
#
from determined import keras
print ("You can ignore WARNING message about nvidia-smi.")
model = keras.load_model_from_checkpoint_path(checkpoint_path)
print("Loaded trained model")

### Making inferences

Now you can use the model to make predictions.

Here, you're going to supply three (3) unlabelled examples (each with the length and width measurements of the flower sepal and petal) to the model. Based on the flower's measurements, the model will predict, for each unlabelled example, the ***likelihood*** that the flower is the given Iris species (the label).
 
The label numbers are mapped to a named representation as:
* 0: Iris setosa
* 1: Iris versicolor
* 2: Iris virginica

In [ ]:
#
# Now you can use the model to make predictions.
# mesasurement is Sepal length, Sepal width, Petal length, Petal width,  
X_new = np.array([[5, 3.9, 2, 0.5], [5, 2.5, 3, 1], [6.9, 3.1, 5.4, 2.1]])
prediction = model(X_new)
print("Let's predict the likelihood that the flower is the given Iris species 0: Iris setosa, 1: Iris versicolor, 2: Iris virginica {}".format(prediction))

**How to interpret the output of the predictions?**

Each prediction is an array of 3 numbers. They represent the model's confidence that the flower corresponds to each of the 3 different species of Iris. Among the 3 numbers, you can see which label has the highest confidence value:

Here you can see that for the first set of measurements, the model predicts with high confidence (+90% probability) that the Iris specie is an Iris Setosa.  

For the second set of measurements, the model predicts the Iris specie is an Iris versicolor (+70% probability), and for the third set of measurements, the model predicts the iris specie is an Iris virginica (+60% probability).

Finally, let's print out the actual numeric value for each unlabelled example that has the highest confidence value:

In [ ]:
print("")
print("The prediction of species for the 3 unlabelled examples is:")
print(np.argmax(prediction,axis=1))

So the model is most confident that the first example is an Iris Setosa, the second is an Iris versicolor and the third is an iris virginica.

> <font color="red"> ***IMPORTANT: When you have finished with the Inferences in JulyterLab on Determined system, please log out from the JupyterLab (File --> Logout) and get back to your local Jupyter Notebook from <ins>"step 10"</ins> to perform some cleanup.*** </font>